Készítsen lekérdezést, amely megjeleníti, hogy szobánként (SZOBA\_FK) hány olyan foglalás történt, ahol a felnőttek száma nagyobb volt a gyermekek számánál!

a. A lekérdezés jelenítse meg a végösszeget is megfelelően jelölve!

In [4]:
SELECT (   
    CASE GROUPING(Szoba_fk)
    WHEN 0 THEN CAST(Szoba_fk AS nvarchar)
    WHEN 1 THEN 'Összesen' END
) AS 'Szoba_FK', 
        Count(*) AS 'Darabszám'
FROM foglalas
WHERE Felnott_szam > gyermek_szam
GROUP BY Rollup(Szoba_fk)

(190 rows affected)

Total execution time: 00:00:00.013

Szoba_FK,Darabszám
1,7
2,2
3,3
4,5
5,2
6,5
7,6
8,5
9,4
10,4


Rangsoroljuk az egyes szobákat a foglalások száma, azon belül férőhely szerint!

- Holtverseny esetén a sorszámok sűrűn kövessék egymást
- Az 1-es számot a legtöbbet foglalt, azon belül legkevesebb férőhelyű szoba kapja
- Csak a szoba azonosítója, a férőhely és a helyezés jelenjen meg

In [26]:
SELECT s.SZOBA_ID, s.ferohely, 
        Dense_RANK() OVER(Partition BY f.szoba_fk
            ORDER BY Count(*), s.Ferohely) AS 'Rank'
FROM Foglalas f JOIN Szoba s ON f.SZOBA_FK = s.SZOBA_ID

: Msg 8120, Level 16, State 1, Line 1
Column 'Szoba.SZOBA_ID' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause.

Total execution time: 00:00:00.002

Kérdezzük le az átlagosnál kevesebb főre történt foglalások adatait!

a. Egy új oszlopban jelenjen meg az ügyfél neve is!

b. Egy másik új oszlop mutassa meg, hogy az ügyfél előző három foglalása átlagosan hány főre történt! Az oszlop neve legyen 'Előző három foglalás átlaga'

c. A listában azon vendégek is jelenjenek meg, akik még nem foglaltak!

In [37]:
SELECT F.*,
        V.NEV,
        AVG(FELNOTT_SZAM + GYERMEK_SZAM) OVER(PARTITION BY V.NEV
            ORDER BY V.NEV
            ROWS BETWEEN 3 PRECEDING AND CURRENT ROW) AS 'eLŐZŐ HÁROM FOGLALÁS ÁTLAGA',
        FELNOTT_SZAM + GYERMEK_SZAM as 'összfő'
FROM Foglalas F LEFT JOIN Vendeg V ON  F.UGYFEL_FK = V.USERNEV
WHERE FELNOTT_SZAM + GYERMEK_SZAM < (
    SELECT AVG(FELNOTT_SZAM + GYERMEK_SZAM)
    FROM Foglalas
)

(147 rows affected)

Total execution time: 00:00:00.012

FOGLALAS_PK,UGYFEL_FK,SZOBA_FK,METTOL,MEDDIG,FELNOTT_SZAM,GYERMEK_SZAM,NEV,eLŐZŐ HÁROM FOGLALÁS ÁTLAGA,összfő
757,david,88,2016-08-05,2016-08-12,1,0,Ambrus Dávid,1,1
923,LASZLOA,123,2016-06-07,2016-06-11,1,0,Antal László,1,1
1005,beata4,22,2016-07-03,2016-07-06,1,0,Bagi Beáta,1,1
1112,beata4,155,2016-10-01,2016-10-05,1,0,Bagi Beáta,1,1
1436,alexandra,35,2017-01-13,2017-01-14,1,0,Bagóczki Alexandra,1,1
1187,eszter2,46,2016-10-23,2016-10-30,1,0,Balogh Eszter,1,1
1171,eszter2,172,2016-10-20,2016-10-25,1,0,Balogh Eszter,1,1
1109,eszter2,110,2016-10-01,2016-10-06,1,0,Balogh Eszter,1,1
1017,eszter2,190,2016-07-08,2016-07-09,1,0,Balogh Eszter,1,1
1480,eszter2,35,2016-12-29,2017-02-04,1,0,Balogh Eszter,1,1
